In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import seaborn as sns
import operator
import matplotlib.pyplot as plt
from statistics import variance
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA, FastICA
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.random_projection import johnson_lindenstrauss_min_dim
from sklearn.random_projection import GaussianRandomProjection
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.metrics import r2_score
from prettytable import PrettyTable
from joblib import Parallel, delayed
import joblib


import warnings
warnings.filterwarnings("ignore")

In [2]:
def final_fun_1(X):
    
    #dropping id
    id = X['ID']
    X = X.drop(columns = ['ID'])
    
    #Removing Outliers
    #X = X.drop(X[data['y'] > 155].index).reset_index(drop = True)
    
    #Removing 'X4' due to Low Variance
    X = X.drop(columns = ['X4'])
    
    #Removing Features with Zero and Same Variance
    
    #Code
    
#     binary_features = X.drop(columns = ['ID', 'X0', 'X1', 'X2', 'X3', 'X5', 'X6', 'X8'])
#     clm_lst = list(binary_features.columns)

#     var_lst = []
#     for ele in clm_lst:
#         var_lst.append(variance(binary_features[ele]))
    
#     var_df = pd.DataFrame()
#     var_df['feature'] = clm_lst
#     var_df['variance'] = var_lst
    
#     ftr_zero_var = []
#     for i in range(var_df.shape[0]):
#         if var_df['variance'][i] == 0:
#             ftr_zero_var.append(var_df['feature'][i])
            
#     index_zero_var = var_df.index[var_df['variance'] == 0].tolist()
#     var_df_upd = var_df.drop(index_zero_var, axis = 0).reset_index(drop = True)

#     ftr_same_var = []
#     for i in range(var_df_upd.shape[0]):
#         for j in range(var_df_upd.shape[0]):
#             if var_df_upd['feature'][i] == var_df_upd['feature'][j]:
#                 continue
#             elif var_df_upd['variance'][i] == var_df_upd['variance'][j]:
#                 if var_df_upd['feature'][i] not in ftr_same_var:
#                     ftr_same_var.append(var_df_upd['feature'][i])
#                 elif var_df_upd['feature'][j] not in ftr_same_var:
#                     ftr_same_var.append(var_df_upd['feature'][j])
#             else:
#                 continue
    
#     X = X.drop(columns = ftr_zero_var + ftr_same_var)
    
    feature_zero_var = joblib.load('FeatureZeroVar.pkl')
    feature_same_var = joblib.load('FeatureSameVar.pkl')
    
    X = X.drop(columns = feature_zero_var + feature_same_var)
    
    #Encoding Categorical Features
    
    def encoding_categorical(data):
        le = LabelEncoder()
        data_le = le.fit_transform(data)
        return data_le
    
    categorical_features = ['X0', 'X1', 'X2', 'X3', 'X5', 'X6', 'X8']
    catg = X[categorical_features]
    
    catg_encoded = pd.DataFrame()

    for feature in categorical_features:
        catg_encoded[feature] = encoding_categorical(catg[feature])
        
    #PCA
    
    binary = X.drop(columns = ['X0', 'X1', 'X2', 'X3', 'X5', 'X6', 'X8'])
    
#     pca = PCA()
#     pca.fit(binary)
#     plt.plot(pca.explained_variance_ratio_.cumsum())
#     plt.xlabel('number of components')
#     plt.ylabel('cumulative explained variance')
    
    pca = joblib.load('PCA.pkl')
    pca_binary = pca.transform(binary)
    pca_binary_df = pd.DataFrame (pca_binary)
    pca_binary_df = pca_binary_df[range(0, 100)]
    
    #SVD
    
#     svd = TruncatedSVD(n_components = 157)
#     svd.fit(binary)
#     plt.plot(svd.explained_variance_ratio_.cumsum())
#     plt.xlabel('number of components')
#     plt.ylabel('cumulative explained variance')
    
    svd = joblib.load('SVD.pkl')
    svd_binary = svd.transform(binary)
    svd_binary_df = pd.DataFrame (svd_binary)
    svd_binary_df = svd_binary_df[range(0, 100)]
    
    #GRP
    
#     n_components = johnson_lindenstrauss_min_dim(n_samples = 4201, eps = 0.9)
#     grp = GaussianRandomProjection(n_components = 206, random_state = 42)
#     grp.fit(binary)
    
    grp = joblib.load('GRP.pkl')
    grp_binary = grp.transform(binary)
    grp_binary_df = pd.DataFrame (grp_binary)
    
    #Interaction
    
#     pf = PolynomialFeatures(interaction_only = True)
#     pf.fit(binary)
#     pf_binary = pf.transform(binary)
#     pf_binary_df = pd.DataFrame(pf_binary)
    
#     #Top 10 Interaction Features
    
#     pf_var = []
#     for feature in pf_binary_df.columns:
#         pf_var.append(variance(pf_binary_df[feature]))     
#     pf_ftr = list(pf_binary_df.columns)
#     ftr_var_dict = {pf_ftr[i]: pf_var[i] for i in range(len(pf_var))}
    
#     pf_ftr_var_df = pd.DataFrame()
#     pf_ftr_var_df['feature'] = pf_binary_df.columns
#     pf_ftr_var_df['variance'] = pf_var
    
#     ftr_zero_var = []
#     for i in range(pf_ftr_var_df.shape[0]):
#         if pf_ftr_var_df['variance'][i] == 0:
#             ftr_zero_var.append(pf_ftr_var_df['feature'][i])
            
#     index_zero_var = pf_ftr_var_df.index[pf_ftr_var_df['variance'] == 0].tolist()
#     pf_ftr_var_df_upd = pf_ftr_var_df.drop(index_zero_var, axis = 0).reset_index(drop = True)
    
#     ftr_same_var = []
#     for i in range(pf_ftr_var_df_upd.shape[0]):
#         for j in range(pf_ftr_var_df_upd.shape[0]):
#             if pf_ftr_var_df_upd['feature'][i] == pf_ftr_var_df_upd['feature'][j]:
#                 continue
#             elif pf_ftr_var_df_upd['variance'][i] == pf_ftr_var_df_upd['variance'][j]:
#                 if pf_ftr_var_df_upd['feature'][i] not in ftr_same_var:
#                     ftr_same_var.append(pf_ftr_var_df_upd['feature'][i])
#                 elif pf_ftr_var_df_upd['feature'][j] not in ftr_same_var:
#                     ftr_same_var.append(pf_ftr_var_df_upd['feature'][j])
#             else:
#                 continue
                
#     ftr_to_drop = ftr_zero_var + ftr_same_var
#     final_pf_binary_df = pf_binary_df.drop(columns = ftr_to_drop)
    
#     pf_var_new = []
#     for feature in final_pf_train_binary_df.columns:
#         pf_var_new.append(variance(final_pf_train_binary_df[feature]))
        
#     pf_ftr_var_df_new = pd.DataFrame()
#     pf_ftr_var_df_new['feature'] = final_pf_binary_df.columns
#     pf_ftr_var_df_new['variance'] = pf_var_new
    
#     pf_ftr = list(final_pf_binary_df.columns)
#     ftr_var_dict = {pf_ftr[i]: pf_var_new[i] for i in range(len(pf_var_new))}
#     sorted_ftr_var_dict = dict( sorted(ftr_var_dict.items(), key=operator.itemgetter(1), reverse=True))
    
    pf = joblib.load('PF.pkl')
    pf_binary = pf.transform(binary)
    pf_binary_df = pd.DataFrame(pf_binary)
    top_10_interaction_features = [12526, 6639, 4899, 3178, 2437, 2483, 6181, 2811, 6854, 4573]
    pf_binary_df = pf_binary_df[top_10_interaction_features]
    
    #Creating dataframes for model with original and new Features
    
    original = pd.concat([catg_encoded, X.drop(columns = ['X0', 'X1', 'X2', 'X3', 'X5', 'X6', 'X8'])], axis = 1, join = 'inner')
    original_all = pd.concat([original, pca_binary_df, svd_binary_df, grp_binary_df, pf_binary_df], axis = 1, join = 'inner')
    
    #Model
    
    dtr = joblib.load('DTR.pkl')
    rfr = joblib.load('RandomForestRegressor.pkl')
    
    pred_dtr = dtr.predict(original)
    pred_rfr = rfr.predict(original_all)
    
    pred = (pred_dtr + pred_rfr)/2
    
    return pred

In [3]:
def final_fun_2(X, y):
    
    #dropping id
    id = X['ID']
    X = X.drop(columns = ['ID'])
    
    #Removing Outliers
    #X = X.drop(X[data['y'] > 155].index).reset_index(drop = True)
    
    #Removing 'X4' due to Low Variance
    X = X.drop(columns = ['X4'])
    
    #Removing Features with Zero and Same Variance
    
    #Code
    
#     binary_features = X.drop(columns = ['ID', 'X0', 'X1', 'X2', 'X3', 'X5', 'X6', 'X8'])
#     clm_lst = list(binary_features.columns)

#     var_lst = []
#     for ele in clm_lst:
#         var_lst.append(variance(binary_features[ele]))
    
#     var_df = pd.DataFrame()
#     var_df['feature'] = clm_lst
#     var_df['variance'] = var_lst
    
#     ftr_zero_var = []
#     for i in range(var_df.shape[0]):
#         if var_df['variance'][i] == 0:
#             ftr_zero_var.append(var_df['feature'][i])
            
#     index_zero_var = var_df.index[var_df['variance'] == 0].tolist()
#     var_df_upd = var_df.drop(index_zero_var, axis = 0).reset_index(drop = True)

#     ftr_same_var = []
#     for i in range(var_df_upd.shape[0]):
#         for j in range(var_df_upd.shape[0]):
#             if var_df_upd['feature'][i] == var_df_upd['feature'][j]:
#                 continue
#             elif var_df_upd['variance'][i] == var_df_upd['variance'][j]:
#                 if var_df_upd['feature'][i] not in ftr_same_var:
#                     ftr_same_var.append(var_df_upd['feature'][i])
#                 elif var_df_upd['feature'][j] not in ftr_same_var:
#                     ftr_same_var.append(var_df_upd['feature'][j])
#             else:
#                 continue
    
#     X = X.drop(columns = ftr_zero_var + ftr_same_var)
    
    feature_zero_var = joblib.load('FeatureZeroVar.pkl')
    feature_same_var = joblib.load('FeatureSameVar.pkl')
    
    X = X.drop(columns = feature_zero_var + feature_same_var)
    
    #Encoding Categorical Features
    
    def encoding_categorical(data):
        le = LabelEncoder()
        data_le = le.fit_transform(data)
        return data_le
    
    categorical_features = ['X0', 'X1', 'X2', 'X3', 'X5', 'X6', 'X8']
    catg = X[categorical_features]
    
    catg_encoded = pd.DataFrame()

    for feature in categorical_features:
        catg_encoded[feature] = encoding_categorical(catg[feature])
        
    #PCA
    
    binary = X.drop(columns = ['X0', 'X1', 'X2', 'X3', 'X5', 'X6', 'X8'])
    
#     pca = PCA()
#     pca.fit(binary)
#     plt.plot(pca.explained_variance_ratio_.cumsum())
#     plt.xlabel('number of components')
#     plt.ylabel('cumulative explained variance')
    
    pca = joblib.load('PCA.pkl')
    pca_binary = pca.transform(binary)
    pca_binary_df = pd.DataFrame (pca_binary)
    pca_binary_df = pca_binary_df[range(0, 100)]
    
    #SVD
    
#     svd = TruncatedSVD(n_components = 157)
#     svd.fit(binary)
#     plt.plot(svd.explained_variance_ratio_.cumsum())
#     plt.xlabel('number of components')
#     plt.ylabel('cumulative explained variance')
    
    svd = joblib.load('SVD.pkl')
    svd_binary = svd.transform(binary)
    svd_binary_df = pd.DataFrame (svd_binary)
    svd_binary_df = svd_binary_df[range(0, 100)]
    
    #GRP
    
#     n_components = johnson_lindenstrauss_min_dim(n_samples = 4201, eps = 0.9)
#     grp = GaussianRandomProjection(n_components = 206, random_state = 42)
#     grp.fit(binary)
    
    grp = joblib.load('GRP.pkl')
    grp_binary = grp.transform(binary)
    grp_binary_df = pd.DataFrame (grp_binary)
    
    #Interaction
    
#     pf = PolynomialFeatures(interaction_only = True)
#     pf.fit(binary)
#     pf_binary = pf.transform(binary)
#     pf_binary_df = pd.DataFrame(pf_binary)
    
#     #Top 10 Interaction Features
    
#     pf_var = []
#     for feature in pf_binary_df.columns:
#         pf_var.append(variance(pf_binary_df[feature]))     
#     pf_ftr = list(pf_binary_df.columns)
#     ftr_var_dict = {pf_ftr[i]: pf_var[i] for i in range(len(pf_var))}
    
#     pf_ftr_var_df = pd.DataFrame()
#     pf_ftr_var_df['feature'] = pf_binary_df.columns
#     pf_ftr_var_df['variance'] = pf_var
    
#     ftr_zero_var = []
#     for i in range(pf_ftr_var_df.shape[0]):
#         if pf_ftr_var_df['variance'][i] == 0:
#             ftr_zero_var.append(pf_ftr_var_df['feature'][i])
            
#     index_zero_var = pf_ftr_var_df.index[pf_ftr_var_df['variance'] == 0].tolist()
#     pf_ftr_var_df_upd = pf_ftr_var_df.drop(index_zero_var, axis = 0).reset_index(drop = True)
    
#     ftr_same_var = []
#     for i in range(pf_ftr_var_df_upd.shape[0]):
#         for j in range(pf_ftr_var_df_upd.shape[0]):
#             if pf_ftr_var_df_upd['feature'][i] == pf_ftr_var_df_upd['feature'][j]:
#                 continue
#             elif pf_ftr_var_df_upd['variance'][i] == pf_ftr_var_df_upd['variance'][j]:
#                 if pf_ftr_var_df_upd['feature'][i] not in ftr_same_var:
#                     ftr_same_var.append(pf_ftr_var_df_upd['feature'][i])
#                 elif pf_ftr_var_df_upd['feature'][j] not in ftr_same_var:
#                     ftr_same_var.append(pf_ftr_var_df_upd['feature'][j])
#             else:
#                 continue
                
#     ftr_to_drop = ftr_zero_var + ftr_same_var
#     final_pf_binary_df = pf_binary_df.drop(columns = ftr_to_drop)
    
#     pf_var_new = []
#     for feature in final_pf_train_binary_df.columns:
#         pf_var_new.append(variance(final_pf_train_binary_df[feature]))
        
#     pf_ftr_var_df_new = pd.DataFrame()
#     pf_ftr_var_df_new['feature'] = final_pf_binary_df.columns
#     pf_ftr_var_df_new['variance'] = pf_var_new
    
#     pf_ftr = list(final_pf_binary_df.columns)
#     ftr_var_dict = {pf_ftr[i]: pf_var_new[i] for i in range(len(pf_var_new))}
#     sorted_ftr_var_dict = dict( sorted(ftr_var_dict.items(), key=operator.itemgetter(1), reverse=True))
    
    pf = joblib.load('PF.pkl')
    pf_binary = pf.transform(binary)
    pf_binary_df = pd.DataFrame(pf_binary)
    top_10_interaction_features = [12526, 6639, 4899, 3178, 2437, 2483, 6181, 2811, 6854, 4573]
    pf_binary_df = pf_binary_df[top_10_interaction_features]
    
    #Creating dataframes for model with original and new Features
    
    original = pd.concat([catg_encoded, X.drop(columns = ['X0', 'X1', 'X2', 'X3', 'X5', 'X6', 'X8'])], axis = 1, join = 'inner')
    original_all = pd.concat([original, pca_binary_df, svd_binary_df, grp_binary_df, pf_binary_df], axis = 1, join = 'inner')
    
    #Model
    
#     x_train_org, x_cv_org, y_train_org, y_cv_org = train_test_split(original, y, test_size = 0.2, random_state = 42)
#     x_train_all, x_cv_all, y_train_all, y_cv_all = train_test_split(original_all, y, test_size = 0.2, random_state = 42)
      
#     dtr = DecisionTreeRegressor(max_depth = 5)
#     dtr.fit(x_train_org, y_train_org)

#     rfr = RandomForestRegressor(n_estimators = 500, max_features = 'auto', max_depth = 3, min_samples_leaf = 1, min_samples_split = 10)
#     rfr.fit(x_train_all, y_train_all)
    
    x_train_org, x_cv_org, y_train_org, y_cv_org = train_test_split(original, y, test_size = 0.2, random_state = 42)
    x_train_all, x_cv_all, y_train_all, y_cv_all = train_test_split(original_all, y, test_size = 0.2, random_state = 42)

    
    dtr = joblib.load('DTR.pkl')
    rfr = joblib.load('RandomForestRegressor.pkl')
    
    pred_dtr = dtr.predict(x_cv_org)
    pred_rfr = rfr.predict(x_cv_all)
    
    pred = (pred_dtr + pred_rfr)/2
    
    r2 = r2_score(y_cv_all, pred)
    
    return r2

In [4]:
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')
y = train['y']
train = train.drop(columns = ['y'])

In [5]:
final_fun_1(test)

array([ 77.51600352, 112.39332834,  78.19243321, ...,  93.94086633,
       112.65364148,  93.94086633])

In [6]:
final_fun_2(train, y)

0.610665245992309